In [4]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import cv2, os

### Note:  You can skip Step 1 and 2 if you do not want to run the evaluation. You can use our `output` folder from [here](https://drive.google.com/drive/u/1/folders/1cMDvOlGF1XWl98dUlTmkPmoREFFFfQ2D) which contains the estimated scene coordinates.


# Step 1: Generate the IMAGE and LABEL list, in the ```input/``` folder


### [UNCOMMENT BELOW CODE]: TO GENERATE LABEL LIST and IMAGE LIST

- make sure the groundtruth labels are downloaded from the links in [main repository](https://github.com/zlthinker/KFNet) and stored in ```inputs/``` folder as ```$folder$-label ```

- make sure the images are downloaded from the 7scenes dataset [website](https://www.microsoft.com/en-us/research/project/rgb-d-dataset-7-scenes/) and saved under ``` inputs/``` as ``` $folder$-input-images```

- The `label_list.txt` and the `image_list.txt` need to be saved in the ``` inputs/``` directory. Also copy the `transform.txt` from ```$folder$-label ``` and paste in ``` inputs/```

In [5]:


# folder = 'head'
# sequence = 'seq-01'

# BIN_DIR = f"./input/{folder}-label/{sequence}/bin"
# N = len(glob(f"{BIN_DIR}/*.bin"))
# files = [f"{BIN_DIR}/{str(i)}.bin"  for i in range(N)]
# np.savetxt('./input/label_list.txt', np.array(files), fmt='%s')
# N = len(glob(f"./input/{folder}-input-images/{sequence}/*.color.png"))
# files = sorted(glob(f"./input/{folder}-input-images/{sequence}/*.color.png"))
# np.savetxt(f'./input/image_list.txt', np.array(files), fmt='%s')

# Step2: Run Evaluation
### use docker in the terminal to load to the workspace and execute evaluation.
- download the models and place in the models folder.
- run the evaluation script using the following command

- Example:
 `python OFlowNet/eval.py --input_folder ./input --output_folder ./output/fire --model_folder ./models/OFlowNet --scene fire`
- note the distance errors
- save the scene coords in the output folder


# Visualize Optical Flow
- specify the scene and the model  

## Functions

In [6]:
def foldercheck(Savepath):
    if(not (os.path.isdir(Savepath))):
        print(Savepath, "  was not present, creating the folder...")
        os.makedirs(Savepath)

def make_colorwheel():
    """
    Generates a color wheel for optical flow visualization as presented in:
        Baker et al. "A Database and Evaluation Methodology for Optical Flow" (ICCV, 2007)
        URL: http://vision.middlebury.edu/flow/flowEval-iccv07.pdf
    Code follows the original C++ source code of Daniel Scharstein.
    Code follows the the Matlab source code of Deqing Sun.
    Returns:
        np.ndarray: Color wheel
    """

    RY = 15
    YG = 6
    GC = 4
    CB = 11
    BM = 13
    MR = 6

    ncols = RY + YG + GC + CB + BM + MR
    colorwheel = np.zeros((ncols, 3))
    col = 0

    # RY
    colorwheel[0:RY, 0] = 255
    colorwheel[0:RY, 1] = np.floor(255*np.arange(0,RY)/RY)
    col = col+RY
    # YG
    colorwheel[col:col+YG, 0] = 255 - np.floor(255*np.arange(0,YG)/YG)
    colorwheel[col:col+YG, 1] = 255
    col = col+YG
    # GC
    colorwheel[col:col+GC, 1] = 255
    colorwheel[col:col+GC, 2] = np.floor(255*np.arange(0,GC)/GC)
    col = col+GC
    # CB
    colorwheel[col:col+CB, 1] = 255 - np.floor(255*np.arange(CB)/CB)
    colorwheel[col:col+CB, 2] = 255
    col = col+CB
    # BM
    colorwheel[col:col+BM, 2] = 255
    colorwheel[col:col+BM, 0] = np.floor(255*np.arange(0,BM)/BM)
    col = col+BM
    # MR
    colorwheel[col:col+MR, 2] = 255 - np.floor(255*np.arange(MR)/MR)
    colorwheel[col:col+MR, 0] = 255
    return colorwheel


def flow_uv_to_colors(u, v, convert_to_bgr=False):
    """
    Applies the flow color wheel to (possibly clipped) flow components u and v.
    According to the C++ source code of Daniel Scharstein
    According to the Matlab source code of Deqing Sun
    Args:
        u (np.ndarray): Input horizontal flow of shape [H,W]
        v (np.ndarray): Input vertical flow of shape [H,W]
        convert_to_bgr (bool, optional): Convert output image to BGR. Defaults to False.
    Returns:
        np.ndarray: Flow visualization image of shape [H,W,3]
    """
    flow_image = np.zeros((u.shape[0], u.shape[1], 3), np.uint8)
    colorwheel = make_colorwheel()  # shape [55x3]
    ncols = colorwheel.shape[0]
    rad = np.sqrt(np.square(u) + np.square(v))
    a = np.arctan2(-v, -u)/np.pi
    fk = (a+1) / 2*(ncols-1)
    k0 = np.floor(fk).astype(np.int32)
    k1 = k0 + 1
    k1[k1 == ncols] = 0
    f = fk - k0
    for i in range(colorwheel.shape[1]):
        tmp = colorwheel[:,i]
        col0 = tmp[k0] / 255.0
        col1 = tmp[k1] / 255.0
        col = (1-f)*col0 + f*col1
        idx = (rad <= 1)
        col[idx]  = 1 - rad[idx] * (1-col[idx])
        col[~idx] = col[~idx] * 0.75   # out of range
        # Note the 2-i => BGR instead of RGB
        ch_idx = 2-i if convert_to_bgr else i
        flow_image[:,:,ch_idx] = np.floor(255 * col)
    return flow_image


def visualize(flow_uv, clip_flow=None, convert_to_bgr=False):
    """
    Expects a two dimensional flow image of shape.
    Args:
        flow_uv (np.ndarray): Flow UV image of shape [H,W,2]
        clip_flow (float, optional): Clip maximum of flow values. Defaults to None.
        convert_to_bgr (bool, optional): Convert output image to BGR. Defaults to False.
    Returns:
        np.ndarray: Flow visualization image of shape [H,W,3]
    """
    assert flow_uv.ndim == 3, 'input flow must have three dimensions'
    assert flow_uv.shape[2] == 2, 'input flow must have shape [H,W,2]'
    
    if clip_flow is not None:
        flow_uv = np.clip(flow_uv, 0, clip_flow)
    u = flow_uv[:,:,0]
    v = flow_uv[:,:,1]
    rad = np.sqrt(np.square(u) + np.square(v))
    rad_max = np.max(rad)
    epsilon = 1e-5
    u = u / (rad_max + epsilon)
    v = v / (rad_max + epsilon)
    return flow_uv_to_colors(u, v, convert_to_bgr)
    
def plotQuivers(image_file1, image_file2, flow_file, thres=100):
    image1 = cv2.imread(image_file1)
    image2 = cv2.imread(image_file2)
    image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
    image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    flow_uv = np.load(flow_file)

    blend_image = cv2.addWeighted(image1, 0.5, image2, 0.5, 0)

    height, width, _ = flow_uv.shape
    # print( flow_uv.shape)
    for h in range(2, height-2, 3):
        for w in range(2, width-2, 3):
            confidence = flow_uv[h, w, 2]
            if confidence < thres:
                continue
            u = flow_uv[h, w, 0]
            v = flow_uv[h, w, 1]

            x = w * 8 + 4
            y = h * 8 + 4
            cv2.line(blend_image, (x, y), (int(x+u*8), int(y+v*8)), (0, 255, 0), 2)
    return blend_image

def visualize_opticalflow(flow):
    confidence = flow[..., 2]        
    flow = flow[..., :2]
    uncertainity = np.dstack((confidence,confidence,confidence))
    flowrgb = visualize(flow)
    return flowrgb, uncertainity


In [ ]:
folder = 'fire'
N = len(glob(f"output/{folder}/flow/*.npy"))-1
flowfiles = [f'output/{folder}/flow/{str(i)}_{str(i+1)}_flow.npy' for i in range(N)]
imagefiles = sorted(glob(f'input/{folder}-input-images/seq-01/*.color.png'))
print(len(imagefiles), len(flowfiles))

for i in range( len(imagefiles)-2 ):
    quiverplot = plotQuivers(imagefiles[i], imagefiles[i+1], flowfiles[i])
    quiverplot=cv2.cvtColor(quiverplot, cv2.COLOR_RGB2BGR)   
    flowrgb, uncertainity = visualize_opticalflow(np.load(flowfiles[i]))
    uncertainity =  cv2.normalize(uncertainity, None, 0, 255, cv2.NORM_MINMAX)

    out = np.vstack(( quiverplot, np.hstack(  (cv2.resize(flowrgb, (flowrgb.shape[1]*4, flowrgb.shape[0]*4)),\
        cv2.resize(uncertainity, (uncertainity.shape[1]*4, uncertainity.shape[0]*4)))  ))) 
    out = out.astype(np.uint8)
    if(i==0):
        h,w = out.shape[:2]
        foldercheck(f'./output/{folder}/')    
        writer = cv2.VideoWriter(f'./output/{folder}_video.mp4', cv2.VideoWriter_fourcc('m', 'p', '4', 'v'), 20, (w, h))    
    writer.write(out)

writer.release()
print("Done stitching")
print(f"Check './output/' {folder}_video.mp4")